In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import time
import gc

model_file = 'model'                   # 不需要修改

label_feat = ['label_5','label_10','label_20', 'label_40', 'label_60']
indexes = ['sym', 'date', 'time']

In [24]:
import glob
import pandas as pd

# 指定路径
path = './zxy_importance_1/'

# 获取路径下的所有文件
files = os.listdir(path)

In [25]:
# 读取每个 CSV 文件并将其存储在一个列表中
dataframes = []
for csv_file in files:
    df = pd.read_csv(os.path.join(path, csv_file), index_col=0)
    dataframes.append(df)


In [26]:
df = pd.concat(dataframes, axis=1)

In [27]:
df['mean'] = df.mean(axis = 1)

In [28]:
df['rank'] = df['mean'].rank(pct = True)

In [29]:
names = df[df['rank'] >= 0.8].index

In [31]:
names = list(names)
names

['ask_price_diff_2_qtlu20',
 'ask_price_diff_2_qtld20',
 'ask_price_diff_2_mean40',
 'ask_price_diff_2_max40',
 'ask_price_diff_2_min40',
 'ask_price_diff_2_qtlu40',
 'ask_price_diff_2_qtld40',
 'ask_price_diff_3_mean40',
 'ask_price_diff_3_max40',
 'ask_price_diff_3_qtlu40',
 'ask_price_diff_4_max5',
 'ask_price_diff_4_min5',
 'ask_price_diff_4_max10',
 'ask_price_diff_4_max20',
 'ask_price_diff_4_qtld20',
 'ask_price_diff_4_mean40',
 'ask_price_diff_4_max40',
 'bid_price_diff_1_mean5',
 'bid_price_diff_1_max5',
 'bid_price_diff_1_qtld5',
 'bid_price_diff_1_mean10',
 'bid_price_diff_1_max10',
 'bid_price_diff_1_mean20',
 'bid_price_diff_1_max20',
 'bid_price_diff_1_qtlu20',
 'bid_price_diff_1_qtld20',
 'bid_price_diff_1_mean40',
 'bid_price_diff_1_max40',
 'bid_price_diff_1_qtlu40',
 'bid_price_diff_2_max5',
 'bid_price_diff_2_min10',
 'bid_price_diff_2_max20',
 'bid_price_diff_2_qtlu20',
 'bid_price_diff_2_mean40',
 'bid_price_diff_2_max40',
 'bid_price_diff_2_qtlu40',
 'bid_price_di

In [32]:
data = pd.read_parquet('./df_train_ordered_1.parquet')
data[names].to_parquet('./select_feature_1.parquet')

ArrowMemoryError: malloc of size 9795100032 failed

In [2]:
def saveDict(data, path):
    pickle.dump(data, open(path, 'wb'), protocol=4)

def fromDict(path):
    print('loading file:', path, end=' ')
    t = time.time()
    data = pickle.loads(open(path,'rb').read())
    print('Time spent:', time.time() - t)
    return data

def load_data(path):
    data = pd.read_parquet(path)
    return data

In [3]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = y_hat.reshape(3,-1).T  # 3 分类
    y_hat = np.argmax(y_hat, axis=-1)
    return 'f1', f1_score(y_true, y_hat, average='macro'), True

In [5]:
input_data = load_data('./df_train_ordered_0.parquet')
input_data = input_data.reset_index(drop = True)
features = input_data.columns
label = load_data('./labels.parquet').sort_values(by=['date', 'time', 'sym'])


X = input_data
Y = label[label_feat]
Y = Y.reset_index(drop = True)
del input_data
gc.collect()

0

In [6]:
from lightgbm import log_evaluation, early_stopping

In [7]:
# 定义LightGBM模型参数
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'seed': 42,
    'metric': 'multi_logloss',  # 设置为'custom'，因为我们使用自定义的评估函数
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 1000,
    'verbose': 1,
    'device_type': 'gpu'
}

# 对每个标签单独训练一个模型
# models = {}
for i, label in enumerate(label_feat):
    callbacks = [log_evaluation(period=20), early_stopping(stopping_rounds=100)]
    print('Begin training', label, '...')
    y = Y[label]

    # 划分训练集和验证集 —— 可修改
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

    # 创建LightGBM数据集
    train_dataset = lgb.Dataset(X_train, label=y_train)
    val_dataset = lgb.Dataset(X_val, label=y_val)

    # del X_train, X_val, y_train, y_val
    # gc.collect()

    # 训练模型
    model = lgb.train(params,
                      train_dataset,
                      valid_sets=[train_dataset, val_dataset],
                      num_boost_round=10000,
                    #   early_stopping_rounds=100,
                      callbacks=callbacks, 
                    #   verbose_eval=100,
                      feval=lgb_f1_score)
    print(model.best_score)
    importance = model.feature_importance(importance_type='split')
    feature_name = model.feature_name()
    importance_df = pd.DataFrame({
        'Feature': feature_name,
        'Importance': importance
    })

    # 保存特征重要性
    importance_path = 'E:\AI_Quant\zxy\importance' + f'{label}_importance_500.csv' # 记得修改
    importance_df.to_csv(importance_path, index=False)

# #     保存模型
#     models[label] = models
    print('End training', label, '...\n\n')
    del X_train, X_val, y_train, y_val
    gc.collect()
# model_path = os.path.join(os.path.dirname(root_path), model_file, os.path.filename(root_path)) + '_model.pkl'
# saveDict(models, model_path)

Begin training label_5 ...
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 102487
[LightGBM] [Info] Number of data points in the train set: 1959020, number of used features: 501
[LightGBM] [Info] Using GPU Device: gfx902, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 501 dense feature groups (941.61 MB) transferred to GPU in 0.711822 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score -1.930704
[LightGBM] [Info] Start training from score -0.344298
[LightGBM] [Info] Start training from score -1.922531
Training until validation scores don't improve for 100 rounds
[20]	training's multi_logloss: 0.657667	training's f1: 0.287175	valid_1's multi_logloss: 0.704566	valid_1's f1: 0.317588
[40]	training's multi_logloss: 0.621414	training's f1: 0.287309	valid_1's multi_logloss: 0.670221	va